In [1]:
from keras.layers import Input, Dense, Dropout
from keras.layers import Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Flatten
import keras.layers as KL
from keras.models import Model
import keras

from training_utils import gen_classifier_dataset

import numpy as np
import json
import time
import os

Using TensorFlow backend.


# Global Config

In [2]:
IMAGE_SHAPE = (200, 200, 1)
NUM_LABELS = 5
BINS_EDGE = np.load("./data/bins_edge.npy")
NUM_CLASSES = len(BINS_EDGE) - 1  

with open('./data/classes_weight.json', 'r') as fp:
    CLASSES_WEIGHT = json.load(fp)

# 1. Model Architect
## 1.1 ResNet Block
Definition of ResNet block is adapted from https://github.com/uzh-rpg/rpg_public_dronet

In [3]:
def convolutional_block(X, num_filters, shape_filters, strides, stage):
    """
    Implementation of convolutional block in Residual network
    
    Input:
        X (tensor): input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
        num_filters (list of 3 ints): list of number of filters
        shape_filters (list of 3 ints): list of filters' shape
        strides (list of 3 ints): list of strides
        stage (int): stage of this convolutional block in the whole ResNet
        
    Output:
        tensor of shape (m, n_H, n_W, n_C)
    """
    
    # retrieve filters shape from filters
    n1, n2, n3 = num_filters
    f1, f2, f3 = shape_filters
    
    # retrieve strides from strides
    s1, s2, s3 = strides
    
    # create name
    bn_name_base = 'bn_' + str(stage) + '_'
    conv_name_base = 'conv_' + str(stage) + '_'
    
    # save value of X
    X_shorcut = X
    
    # First component of the main path
    X = BatchNormalization(name=bn_name_base + 'a')(X)
    X = Activation('relu', name="feature_map_%d" % (stage - 1))(X)
    X = Conv2D(n1, (f1, f1), strides=[s1, s1], padding='same',
               name=conv_name_base + 'a')(X)
    
    # Second component of the main path
    X = BatchNormalization(name=bn_name_base + 'b')(X)
    X = Activation('relu')(X)
    X = Conv2D(n2, (f2, f2), strides=[s2, s2], padding='same',
               name=conv_name_base + 'b')(X)
    
    # Short-cut
    X_shorcut = Conv2D(n3, (f3, f3), strides=[s3, s3], padding='same', 
                       name=conv_name_base + 'c')(X_shorcut)
    
    X = KL.merge.add([X, X_shorcut])
    
    return X


## 1.4 Create Model

In [4]:
def full_model(input_shape):
    """
    Define encoder architecture as ResNet8
    
    Input:
        input_shape (list of ints): shape of input image [n_H, n_W, n_C]
        
    Output:
        model: a Model instance
    """
    
    # Input
    X_input = Input(shape=input_shape)
    
    # Apply 1st convolution & max pooling on input
    X = Conv2D(32, (5, 5), strides=[2,2], padding='same', name='conv_0')(X_input)
    X = MaxPooling2D(pool_size=(3, 3), strides=[2,2])(X) 
    
    # First convolutional block
    X = convolutional_block(X, [32, 32, 32], [3, 3, 1], [2, 1, 2], stage=1)
    
    # Second convolutional block
    X = convolutional_block(X, [64, 64, 64], [3, 3, 1], [2, 1, 2], stage=2)
    
    # Third convolutional block
    X = convolutional_block(X, [128, 128, 128], [3, 3, 1], [2, 1, 2], stage=3)
    
    # Output layer of resnet-8
    X = Flatten()(X)
    X = Activation('relu')(X)
    
    # extract feature vector
    X_feature = Dropout(0.5)(X)
    
    # extract feature vector
    X_feature = Dropout(0.5)(X)
    
    # apply classifier head
    y = []
    for i in range(NUM_LABELS):
        # apply classifier body
        X_body = Dense(800, activation='relu')(X_feature)
        X_body_1 = Dropout(0.5)(X_body)
        out = Dense(NUM_CLASSES, activation='softmax', name="head_%d" % i)(X_body_1)
        y.append(out)
    
    model = Model(inputs=[X_input], outputs=y)    
    return model

In [5]:
model = full_model(IMAGE_SHAPE)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__

In [6]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=['acc'])

# 2. Training

In [ ]:
train_path_X = "./data/CH2_training_X.npy"
train_path_y = "./data/CH2_training_y.npy"
val_path_X = "./data/CH2_validation_X.npy"
val_path_y = "./data/CH2_validation_y.npy"

In [7]:
def load_dataset(path_to_dataset_X, path_to_dataset_y):
    """
    Load train/validation set from .npy file
    Input:
        path_to_dataset (str)
    Output:
        X (np.ndarray): shape (num_samples, image_height, image_width, 1) 
        y (list): each element is a np.ndarray, shape (num_samples, num_classes)
    """
    X = np.load(path_to_dataset_X)
    y_tensor = np.load(path_to_dataset_y)
    y = [y_tensor[i, :, :]for i in range(NUM_LABELS)]
    return X, y


gen_param = {'num_classes': NUM_CLASSES, 
             'num_labels': NUM_LABELS, 
             'bins_edge': BINS_EDGE, 
             'image_shape': IMAGE_SHAPE, 
             'num_samples': None, 
             'data_root_dir': "./data/training_data/center/", # path to folder contained images
             'flip_prob': 0.5}

if os.path.isfile(train_path_X):
    print("Load dataset")
    X_train, y_train = load_dataset(train_path_X, train_path_y)
else:
    print("Generate dataset")
    X_train, y_train = gen_classifier_dataset("./data/CH2_training.csv", **gen_param)
    # save data file for future use
    np.save('./data/CH2_training_X.npy', X_train)
    np.save('./data/CH2_training_y.npy', y_train)

if os.path.isfile(val_path_X):
    X_val, y_val = load_dataset(val_path_X, val_path_y)
else:
    X_val, y_val = gen_classifier_dataset("./data/CH2_validation.csv", **gen_param)
    np.save('./data/CH2_validation_X.npy', X_val)
    np.save('./data/CH2_validation_y.npy', y_val)
    

In [8]:
batch_size = 200

time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './logs/' + time_str

tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

early_stop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              patience=3,
                                              restore_best_weights=True)

history = model.fit(x=X_train,
                  y=y_train,
                  epochs=50,
                  validation_data=(X_val, y_val),
                  class_weight=CLASSES_WEIGHT,
                  initial_epoch=0,
                  shuffle=True,
                  batch_size=batch_size,
                  callbacks=[tb_callback, early_stop_cb])

Instructions for updating:
Use tf.cast instead.
Train on 18183 samples, validate on 2021 samples
Epoch 1/50
18183/18183 [==============================] - 48s 3ms/step - loss: 75.2002 - head_0_loss: 7.2896 - head_1_loss: 7.5561 - head_2_loss: 7.1665 - head_3_loss: 7.2296 - head_4_loss: 7.9905 - head_5_loss: 7.8807 - head_6_loss: 7.3873 - head_7_loss: 8.2520 - head_8_loss: 7.2345 - head_9_loss: 7.2135 - head_0_acc: 0.1900 - head_1_acc: 0.1557 - head_2_acc: 0.1918 - head_3_acc: 0.1902 - head_4_acc: 0.1355 - head_5_acc: 0.1595 - head_6_acc: 0.1828 - head_7_acc: 0.1283 - head_8_acc: 0.1971 - head_9_acc: 0.1896 - val_loss: 29.5784 - val_head_0_loss: 2.9701 - val_head_1_loss: 2.9698 - val_head_2_loss: 2.9502 - val_head_3_loss: 2.9412 - val_head_4_loss: 2.9500 - val_head_5_loss: 2.9544 - val_head_6_loss: 2.9581 - val_head_7_loss: 2.9746 - val_head_8_loss: 2.9468 - val_head_9_loss: 2.9632 - val_head_0_acc: 0.2237 - val_head_1_acc: 0.2246 - val_head_2_acc: 0.2237 - val_head_3_acc: 0.2172 - val_

KeyboardInterrupt: 

# 3. Plot Training Result

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# 4. Save model

In [ ]:
# serialize model to HDF5
model.save(log_dir + "/s1p5_model_%s.h5" % time_str)